# Multi-omics integration & machine learning (toy example)

Obiettivi del notebook:
- Simulare un piccolo dataset **multi-omics** (trascrittomica + metabolomica) con esposizione ambientale.
- Mostrare un esempio di **early integration** (concatenazione delle matrici).
- Esplorare i dati con **PCA**.
- Esempio di integrazione basata su **correlazioni gene–metabolita**.
- Addestrare un semplice **Random Forest** per classificare il livello di esposizione.

_Tutti i dati sono simulati per scopi didattici._

In [ ]:
from __future__ import annotations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

%matplotlib inline
np.random.seed(8)

## 1. Simulazione di un dataset multi-omics + esposizione

Immaginiamo:
- 60 campioni totali (es. individui o organismi).
- 30 **basso inquinamento** (CTRL) e 30 **alta esposizione** (HIGH).
- 100 geni (trascrittomica) e 50 metaboliti (metabolomica).
- Un piccolo sottoinsieme di geni/metaboliti è davvero associato all'esposizione.


In [ ]:
n_samples = 60
n_ctrl = 30
n_high = 30
n_genes = 100
n_mets = 50

sample_ids = [f"S{i+1:03d}" for i in range(n_samples)]
exposure = np.array([0]*n_ctrl + [1]*n_high)  # 0 = low, 1 = high
exposure_labels = np.where(exposure == 0, "LOW", "HIGH")
exposure_series = pd.Series(exposure_labels, index=sample_ids, name="ExposureGroup")

# Trascrittomica: log2(expression) ~ N(0, 1)
X_rna = np.random.normal(loc=0.0, scale=1.0, size=(n_samples, n_genes))
# Metabolomica: log10(intensità) ~ N(5, 0.5)
X_met = np.random.normal(loc=5.0, scale=0.5, size=(n_samples, n_mets))

# Geni/metaboliti realmente associati all'esposizione
signal_genes = np.arange(0, 10)        # primi 10 geni
signal_mets = np.arange(0, 8)          # primi 8 metaboliti

for g in signal_genes:
    X_rna[exposure == 1, g] += 1.2     # up-regolati nel gruppo HIGH
for m in signal_mets:
    X_met[exposure == 1, m] += 0.8     # intensità aumentata nel gruppo HIGH

gene_names = [f"G_{i+1:03d}" for i in range(n_genes)]
met_names = [f"M_{i+1:03d}" for i in range(n_mets)]

rna_df = pd.DataFrame(X_rna, index=sample_ids, columns=gene_names)
met_df = pd.DataFrame(X_met, index=sample_ids, columns=met_names)

rna_df.head()

## 2. QC qualitativo: distribuzione delle intensità per omica

Guardiamo rapidamente la distribuzione delle espressioni geniche e delle intensità metaboliche.

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.hist(rna_df.values.ravel(), bins=30)
plt.xlabel("log2 expression (RNA)")
plt.ylabel("Conteggio")
plt.title("Distribuzione globale RNA")

plt.subplot(1,2,2)
plt.hist(met_df.values.ravel(), bins=30)
plt.xlabel("log10 intensità (metaboliti)")
plt.ylabel("Conteggio")
plt.title("Distribuzione globale metaboliti")

plt.tight_layout()
plt.show()

## 3. Early integration: concatenare le matrici omiche

Prima di concatenare, è buona pratica riportare le feature su scale comparabili (z-score per colonna).

In [ ]:
def zscore_df(df: pd.DataFrame) -> pd.DataFrame:
    return (df - df.mean(axis=0)) / df.std(axis=0, ddof=1)

rna_z = zscore_df(rna_df)
met_z = zscore_df(met_df)

# Concatenazione orizzontale (early integration)
multi_omics_df = pd.concat([rna_z.add_prefix("RNA_"), met_z.add_prefix("MET_")], axis=1)
multi_omics_df.head()

## 4. PCA sull'insieme integrato

Applichiamo una PCA (via SVD) sui dati integrati per vedere se i campioni LOW vs HIGH tendono a separarsi nelle prime componenti.

In [ ]:
# Matrice campioni x feature
X = multi_omics_df.values
# Centro le feature (già z-scored, ma centriamo ancora per sicurezza)
X_centered = X - X.mean(axis=0, keepdims=True)

# SVD
U, S, Vt = np.linalg.svd(X_centered, full_matrices=False)
PCs = U[:, :2] * S[:2]
pca_df = pd.DataFrame(PCs, index=sample_ids, columns=["PC1", "PC2"])
pca_df.head()

### Plot PC1 vs PC2 colorando per gruppo di esposizione

In [ ]:
plt.figure(figsize=(5,4))
for grp, color in [("LOW", "blue"), ("HIGH", "red")]:
    idx = (exposure_series == grp)
    plt.scatter(pca_df.loc[idx, "PC1"], pca_df.loc[idx, "PC2"], label=grp)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA su dati multi-omics integrati")
plt.legend()
plt.tight_layout()
plt.show()

Si può discutere in aula:
- quanto i gruppi siano separati nelle prime componenti,
- il ruolo del numero di feature e del rapporto feature/campioni.

## 5. Integrazione basata su correlazioni gene–metabolita

Come esempio, prendiamo un gene "esposto" e vediamo quali metaboliti sono più correlati ad esso.

In [ ]:
gene_example = "G_001"  # uno dei primi geni, che abbiamo reso dipendente da esposizione
g_vals = rna_df[gene_example]
corrs = []
for met in met_names:
    r, pval = stats.pearsonr(g_vals.values, met_df[met].values)
    corrs.append({"metabolite": met, "r": r, "pval": pval})
corr_df = pd.DataFrame(corrs).sort_values("r", ascending=False)
corr_df.head(10)

Possiamo anche fare uno scatter plot gene vs metabolita più correlato.

In [ ]:
top_met = corr_df.iloc[0]["metabolite"]
plt.scatter(g_vals, met_df[top_met], c=exposure, cmap="coolwarm")
plt.xlabel(f"Espressione {gene_example}")
plt.ylabel(f"Intensità {top_met}")
plt.title(f"Correlazione {gene_example} vs {top_met}")
plt.tight_layout()
plt.show()

## 6. Modello di machine learning supervisionato (Random Forest)

Obiettivo: classificare i campioni in **LOW** vs **HIGH** esposizione usando tutte le feature multi-omics integrate.

Workflow:
1. Split train/test.
2. Addestramento di un `RandomForestClassifier`.
3. Valutazione con accuracy, matrice di confusione e ROC-AUC.


In [ ]:
X = multi_omics_df.values
y = exposure  # 0 = LOW, 1 = HIGH

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

print("Accuracy:", round(acc, 3))
print("ROC-AUC:", round(auc, 3))
print("Confusion matrix:\n", cm)

### ROC curve

Visualizziamo la curva ROC e l'area sotto la curva.

In [ ]:
fpr, tpr, thr = roc_curve(y_test, y_proba)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label=f"ROC AUC = {auc:.2f}")
plt.plot([0,1], [0,1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC curve (Random Forest)")
plt.legend()
plt.tight_layout()
plt.show()

## 7. Interpretazione del modello: feature importance

Estraiamo le feature più importanti secondo la Random Forest e vediamo se sono geni o metaboliti.

In [ ]:
feature_names = multi_omics_df.columns
importances = clf.feature_importances_
imp_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances,
})
imp_df = imp_df.sort_values("importance", ascending=False)
imp_df.head(15)

Possiamo separare l'importanza aggregata per layer (RNA vs MET).

In [ ]:
imp_df["layer"] = imp_df["feature"].apply(lambda x: x.split("_")[0])
imp_by_layer = imp_df.groupby("layer")["importance"].sum()
imp_by_layer

In [ ]:
plt.bar(imp_by_layer.index, imp_by_layer.values)
plt.ylabel("Somma importanza RF")
plt.title("Contributo complessivo dei layer (RNA vs MET)")
plt.show()

In una discussione in aula si può collegare questo a:
- quali livelli omici sembrano più informativi per la classificazione,
- quali geni/metaboliti emergono come candidati biomarcatori,
- rischi di overfitting quando #feature ≫ #campioni.

Questo notebook implementa un flusso didattico coerente con la Lecture 8:
- preprocessing e z-score,
- early integration,
- PCA,
- correlazioni inter-omiche,
- machine learning supervisionato + interpretazione delle feature.